In [1]:
from Kaggle_Titanic_utls import *
import warnings
warnings.filterwarnings('ignore')

In [2]:
PATH_DATA = Path('.')

Let's import the data and do some preprocessing

In [3]:
# Import data
data = pd.read_csv(PATH_DATA / "train.csv")
test = pd.read_csv(PATH_DATA / 'test.csv')

In [4]:
data.shape, test.shape

((891, 12), (418, 11))

In [5]:
data.head(2)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C


In [6]:
test.head(2)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S


In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [8]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    object 
 4   Age          332 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         417 non-null    float64
 9   Cabin        91 non-null     object 
 10  Embarked     418 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB


In [9]:
data['Survived'].sum()/len(data)

0.3838383838383838

The classes are sufficiently balanced - it makes sense to consider accuracy as performance metrics.

In [10]:
#data.dropna(subset=['Embarked'], inplace=True)

In [11]:
labels = data[['Survived']]
data.drop(['Cabin', 'Ticket', 'PassengerId', 'Survived'], axis=1, inplace=True)

test_ids = test.PassengerId
test.drop(['Cabin', 'Ticket', 'PassengerId'], axis=1, inplace=True)

In [12]:
data.corr().style.background_gradient(cmap ='cool')\
        .set_properties(**{'font-size': '15px'}) 

,Pclass,Age,SibSp,Parch,Fare
Pclass,1.000000,-0.369226,0.083081,0.018443,-0.549500
Age,-0.369226,1.000000,-0.308247,-0.189119,0.096067
SibSp,0.083081,-0.308247,1.000000,0.414838,0.159651
Parch,0.018443,-0.189119,0.414838,1.000000,0.216225
Fare,-0.549500,0.096067,0.159651,0.216225,1.000000


We can extract the title of each passenger from the name as the former likely bears more significance than the latter

In [13]:
title = get_title(data)
test_title = get_title(test)

In [14]:
title.value_counts()

Mr              517
Miss            182
Mrs             125
Master           40
Dr                7
Rev               6
Major             2
Mlle              2
Col               2
Ms                1
Sir               1
the Countess      1
Don               1
Jonkheer          1
Lady              1
Mme               1
Capt              1
Name: Name, dtype: int64

In [15]:
data['Title'] = title
data.drop('Name', axis=1, inplace=True)

test['Title'] = test_title
test.drop('Name', axis=1, inplace=True)

Decide whether to use passengers titles or drop these also

In [16]:
no_title = True
if (no_title):
    try:
        data.drop('Title', axis=1, inplace=True)
        test.drop('Title', axis=1, inplace=True)
    except KeyError:
        print('Title is not present')

In [17]:
labels.shape, data.shape

((891, 1), (891, 7))

In [19]:
data.columns

Index(['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked'], dtype='object')

## Preprocessing

Here we impute numerical and categorical features with median and mode, respectively, and then we transform categorical features using one-hot encoding

In [20]:
#data_num, data_cat = get_features_by_type(data)

In [21]:
numeric_features = ['Age', 'Fare', 'SibSp', 'Parch']
numeric_transformer = Pipeline(steps=[
    ('imputer', KNNImputer()),
    ('scaler', StandardScaler())])

categorical_features = ['Embarked', 'Sex', 'Pclass']
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

data_ = preprocessor.fit_transform(data)

In [22]:
test_ = preprocessor.transform(test)

In [23]:
data_.shape, test_.shape

((891, 13), (418, 13))

## Baseline models

We look at various baseline models to check which ones give the best score in and out of sample

In [24]:
models = {'knn': KNeighborsClassifier(), 'LR': LogisticRegression(), 
         'tree': DecisionTreeClassifier(), 'GNB': GaussianNB(),
         'perceptron': Perceptron(), 'SVC': SVC()}

for i, j in models.items():
    scores = model_trial(data_, labels, j)
    print(color.BOLD + color.RED + color.UNDERLINE + f'{i} scores' + color.END + f': {scores}\n\n')

knn scores: {'train_score': 0.8622754491017964, 'test_score': 0.7847533632286996}


LR scores: {'train_score': 0.8083832335329342, 'test_score': 0.7937219730941704}


tree scores: {'train_score': 0.9865269461077845, 'test_score': 0.7847533632286996}


GNB scores: {'train_score': 0.6736526946107785, 'test_score': 0.7174887892376681}


perceptron scores: {'train_score': 0.7664670658682635, 'test_score': 0.7937219730941704}


SVC scores: {'train_score': 0.8413173652694611, 'test_score': 0.8251121076233184}




Perceptron and Naive Bayes have a poor accuracy, whereas Decision Trees are clearly overfitting to the training set. SVC, Logistic Regression and KNN perform reasonably well in and out of sample (random guessing would yield around $62 \%$ accuracy).

## Grid Search

We can now take the best models and preprocessing strategies and grid search the best parameters using cross validation.

grid search: knn

In [25]:
param_grid = {'weights': ['uniform', 'distance'], 
               'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'], 
               'n_neighbors': np.linspace(1, 15, 15).astype(int)}

In [26]:
gscv = GridSearchCV(models['knn'], cv=4, n_jobs=-3, param_grid=param_grid, scoring='accuracy', verbose=1)

In [29]:
%%time
gscv.fit(data_, labels)

Fitting 4 folds for each of 120 candidates, totalling 480 fits
CPU times: user 300 ms, sys: 45 ms, total: 345 ms
Wall time: 1.39 s


GridSearchCV(cv=4, estimator=KNeighborsClassifier(), n_jobs=-3,
             param_grid={'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
                         'n_neighbors': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15]),
                         'weights': ['uniform', 'distance']},
             scoring='accuracy', verbose=1)

In [30]:
gscv.best_estimator_, gscv.best_params_

(KNeighborsClassifier(algorithm='ball_tree', n_neighbors=12),
 {'algorithm': 'ball_tree', 'n_neighbors': 12, 'weights': 'uniform'})

In [31]:
model = gscv.best_estimator_
name_model = 'knn_BE'
try_model(data_, labels, model, name_model)

KNeighborsClassifier(algorithm='ball_tree', n_neighbors=12)
model: knn_BE
train score: 0.8383
test score: 0.7758


In [32]:
create_subfile_titanic(test_, test_ids, model, name_model, PATH_DATA)

0

In [36]:
!kaggle competitions submit -c titanic -f "submission_{name_model}_titanic.csv" -m 'submission of {model}'

100%|████████████████████████████████████████| 2.77k/2.77k [00:03<00:00, 741B/s]
Successfully submitted to Titanic - Machine Learning from Disaster

grid search: lr

In [33]:
param_grid = [{'penalty': ['l1'], 'solver': ['liblinear', 'saga'], 
                'C': np.logspace(-2, 2, 5, 10)},
               {'penalty': ['l2'], 'solver': ['newton-cg', 'lbfgs', 'sag', 'saga'],
                'C': np.logspace(-2, 2, 5, 10)},
               {'penalty': ['elasticnet'], 'solver': ['saga'], 
                'l1_ratio': np.arange(0.1, 1, 0.1), 'C': np.logspace(-2, 2, 5, 10)},
                {'penalty': [None], 'solver': ['newton-cg', 'lbfgs', 'sag', 'saga']}]

In [34]:
gscv = GridSearchCV(models['LR'], cv=4, n_jobs=-3, param_grid=param_grid, scoring='accuracy', verbose=1)

In [35]:
%%time
gscv.fit(data_, labels)

Fitting 4 folds for each of 79 candidates, totalling 316 fits
CPU times: user 332 ms, sys: 35.7 ms, total: 367 ms
Wall time: 1.2 s


GridSearchCV(cv=4, estimator=LogisticRegression(), n_jobs=-3,
             param_grid=[{'C': array([1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02]),
                          'penalty': ['l1'], 'solver': ['liblinear', 'saga']},
                         {'C': array([1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02]),
                          'penalty': ['l2'],
                          'solver': ['newton-cg', 'lbfgs', 'sag', 'saga']},
                         {'C': array([1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02]),
                          'l1_ratio': array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]),
                          'penalty': ['elasticnet'], 'solver': ['saga']},
                         {'penalty': [None],
                          'solver': ['newton-cg', 'lbfgs', 'sag', 'saga']}],
             scoring='accuracy', verbose=1)

In [36]:
gscv.best_estimator_, gscv.best_params_

(LogisticRegression(C=10.0, penalty='l1', solver='liblinear'),
 {'C': 10.0, 'penalty': 'l1', 'solver': 'liblinear'})

In [37]:
model = gscv.best_estimator_
name_model = 'lr_BE'
try_model(data_, labels, model, name_model)

LogisticRegression(C=10.0, penalty='l1', solver='liblinear')
model: lr_BE
train score: 0.8204
test score: 0.7892


In [38]:
create_subfile_titanic(test_, test_ids, model, name_model, PATH_DATA)

0

In [ ]:
!kaggle competitions submit -c titanic -f "submission_{name_model}_titanic.csv" -m 'submission of {model}'

grid search: svc

In [39]:
param_grid = {'C': np.logspace(-2, 2, 5, 10),
               'kernel': ['linear', 'rbf', 'poly'],
              'degree': np.linspace(2, 5, 5).astype(int)}

In [40]:
gscv = GridSearchCV(models['SVC'], cv=4, n_jobs=-3, param_grid=param_grid, scoring='accuracy', verbose=1)

In [41]:
%%time
gscv.fit(data_, labels)

Fitting 4 folds for each of 75 candidates, totalling 300 fits
CPU times: user 266 ms, sys: 33.6 ms, total: 299 ms
Wall time: 3.01 s


GridSearchCV(cv=4, estimator=SVC(), n_jobs=-3,
             param_grid={'C': array([1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02]),
                         'degree': array([2, 2, 3, 4, 5]),
                         'kernel': ['linear', 'rbf', 'poly']},
             scoring='accuracy', verbose=1)

In [42]:
gscv.best_estimator_, gscv.best_params_

(SVC(degree=2), {'C': 1.0, 'degree': 2, 'kernel': 'rbf'})

In [43]:
model = gscv.best_estimator_
name_model = 'svc_BE'
try_model(data_, labels, model, name_model)

SVC(degree=2)
model: svc_BE
train score: 0.8488
test score: 0.7982


Support Vector Classifier with a quadratic polynomial kernel and strength of the regularization equal to $C=1$ yields the best accuracy after grid-searching parameters. 

In [44]:
create_subfile_titanic(test_, test_ids, model, name_model, PATH_DATA)

0

In [ ]:
!kaggle competitions submit -c titanic -f "submission_{name_model}_titanic.csv" -m 'submission of {model}'

## Ensemble Methods

Let's now use some ensemble methods on this dataset. We will first try with hard and soft voting classifiers, and then with Random Forest and Extra Trees

Random Forest

In [45]:
model = RandomForestClassifier(random_state=100)
name_model = 'rf_titanic'
save_model = 'no'
try_model(data_, labels, model, name_model, dump_model=save_model)

RandomForestClassifier(random_state=100)
model: rf_titanic
train score: 0.9850
test score: 0.8161


In [46]:
create_subfile_titanic(test_, test_ids, model, name_model, PATH_DATA)

0

In [51]:
!kaggle competitions submit -c titanic -f \
"submission_{name_model}_titanic.csv" -m \
'submission of {name_model}'

100%|████████████████████████████████████████| 2.77k/2.77k [00:02<00:00, 972B/s]
Successfully submitted to Titanic - Machine Learning from Disaster

Extra Trees

In [47]:
model = ExtraTreesClassifier(random_state=10)
name_model = 'ext_titanic'
save_model = 'no'
try_model(data_, labels, model, name_model, dump_model=save_model)

ExtraTreesClassifier(random_state=10)
model: ext_titanic
train score: 0.9865
test score: 0.7982


In [48]:
create_subfile_titanic(test_, test_ids, model, name_model)

0

### Grid Search

## grid search parameters for ensemble methods?

## Hard and Soft Voting Classifiers

The dataset seems to be very simple, so it make sense to train several weak learners and combine them using soft or hard voting classifiers. Let's see how these two strategies perform.

In [49]:
model = VotingClassifier(estimators=[('svc', SVC(kernel='poly')), \
('lr', LogisticRegression()), ('dt', DecisionTreeClassifier()),\
('gnb', GaussianNB()), ('knn', KNeighborsClassifier())], \
voting='hard', n_jobs=-3, verbose=True)

name_model = 'hv_clf_titanic'
try_model(data_, labels, model, name_model, dump_model='no')

VotingClassifier(estimators=[('svc', SVC(kernel='poly')),
                             ('lr', LogisticRegression()),
                             ('dt', DecisionTreeClassifier()),
                             ('gnb', GaussianNB()),
                             ('knn', KNeighborsClassifier())],
                 n_jobs=-3, verbose=True)
model: hv_clf_titanic
train score: 0.8713
test score: 0.7892


In [50]:
model = VotingClassifier(estimators=[('svc', SVC(probability=True)), \
('lr', LogisticRegression()), ('dt', DecisionTreeClassifier()),\
('gnb', GaussianNB()), ('knn', KNeighborsClassifier())], \
voting='soft', n_jobs=-3, verbose=True)

name_model = 'sv_clf_titanic'
try_model(data_, labels, model, name_model, dump_model='no')

VotingClassifier(estimators=[('svc', SVC(probability=True)),
                             ('lr', LogisticRegression()),
                             ('dt', DecisionTreeClassifier()),
                             ('gnb', GaussianNB()),
                             ('knn', KNeighborsClassifier())],
                 n_jobs=-3, verbose=True, voting='soft')
model: sv_clf_titanic
train score: 0.8937
test score: 0.7803


In [51]:
create_subfile_titanic(test_, test_ids, model, name_model)

0

In [82]:
!kaggle competitions submit -c titanic -f \
"submission_{name_model}_titanic.csv" -m \
'submission of {name_model}'

100%|██████████████████████████████████████| 2.77k/2.77k [00:02<00:00, 1.02kB/s]
Successfully submitted to Titanic - Machine Learning from Disaster

## Boosting

### Gradient Boosting

In [52]:
%%time
param_grid = {'n_estimators': np.arange(10, 101, 5)}
clf = GradientBoostingClassifier()

gscv = GridSearchCV(clf, scoring='accuracy', cv=10,\
param_grid=param_grid, verbose=1, n_jobs=-3)

gscv.fit(data_, labels)

Fitting 10 folds for each of 19 candidates, totalling 190 fits
CPU times: user 234 ms, sys: 56.5 ms, total: 291 ms
Wall time: 2.86 s


GridSearchCV(cv=10, estimator=GradientBoostingClassifier(), n_jobs=-3,
             param_grid={'n_estimators': array([ 10,  15,  20,  25,  30,  35,  40,  45,  50,  55,  60,  65,  70,
        75,  80,  85,  90,  95, 100])},
             scoring='accuracy', verbose=1)

In [53]:
gscv.best_score_

0.8238202247191012

In [54]:
model, name_model = gscv.best_estimator_, 'xgb_BE_titanic'

In [55]:
model.score(data_, labels)

0.8473625140291807

In [56]:
assert create_subfile_titanic(test_,\
test_ids, model, name_model) == 0, 'something wrong in creating submission file'

In [57]:
!kaggle competitions submit -c titanic -f \
"submission_{name_model}_titanic.csv" -m \
'submission of {name_model}'

100%|████████████████████████████████████████| 2.77k/2.77k [00:04<00:00, 681B/s]
Successfully submitted to Titanic - Machine Learning from Disaster

### Adaboost

In [58]:
%%time
param_grid = {'n_estimators': np.arange(10, 101, 5)}
clf = AdaBoostClassifier()

gscv = GridSearchCV(clf, scoring='accuracy', cv=10,\
param_grid=param_grid, verbose=1, n_jobs=-3)

gscv.fit(data_, labels)

Fitting 10 folds for each of 19 candidates, totalling 190 fits
CPU times: user 391 ms, sys: 21.6 ms, total: 412 ms
Wall time: 4.8 s


GridSearchCV(cv=10, estimator=AdaBoostClassifier(), n_jobs=-3,
             param_grid={'n_estimators': array([ 10,  15,  20,  25,  30,  35,  40,  45,  50,  55,  60,  65,  70,
        75,  80,  85,  90,  95, 100])},
             scoring='accuracy', verbose=1)

In [59]:
gscv.best_score_

0.8126342072409489

In [60]:
model, name_model = gscv.best_estimator_, 'adab_BE_titanic'

In [61]:
assert create_subfile_titanic(test_,\
test_ids, model, name_model) == 0, 'something wrong in creating submission file'

In [62]:
!kaggle competitions submit -c titanic -f \
"submission_{name_model}_titanic.csv" -m \
'submission of {name_model}'

100%|████████████████████████████████████████| 2.77k/2.77k [00:04<00:00, 616B/s]
Successfully submitted to Titanic - Machine Learning from Disaster